# The Data Science Process

This is intended to be a follow-up script for my talk entitled [Cleaning is Half the Battle:  Launching a Data Science Project](http://www.catallaxyservices.com/presentations/datascience/).  I will repeat some of the presentation in this notebook, but this notebook is not designed to replace the presentation itself.

Over the course of this notebook, we will analyze Data Professional salaries based on the [2018 Data Professionals Salary Survey](https://www.brentozar.com/archive/2018/01/2018-data-professionals-salary-survey-results/).  We will use this data as the centerpiece of an implementation of the [Microsoft Team Data Science Process](https://docs.microsoft.com/en-us/azure/machine-learning/team-data-science-process/overview).

The Team Data Science Process has five major steps:
1. Business Understanding
2. Data Processing
3. Modeling
4. Deployment
5. Repeat

## Business Understanding

Before we begin working, we need to understand the problem.  In our scenario, we work for Data Platform Specialists, a company dedicated to providing DBAs and other data platform professionals with valuable market knowledge. We have come into possession of a survey of data professionals and want to build insights that we can share with our client base.  Our higher-ups want us to analyze this survey and see how we can make good use of it.

During initial brainstorming, we might come up with questions like:
* How much money does a DBA make?
* Which category of DBA (junior, mid-level, senior) does this particular work?
* How can we segment the DBAs in our survey?
* Is this many hours per week weird?
* Which option should I choose as a career path? DBA? Data science? BI?

These questions could potentially be interesting, and the specific form of the question will help guide our analysis.  For example, "how much" questions typically lead to regression, whereas "which category" questions are indicative of a classification problem.  Segmentation questions where we do not know the classes beforehand is a classic example of a clustering problem, and the final two questions are anomaly detection and recommendation, respectively.

Narrowing this down with our champion and other stakeholders, we can get to the following question which we will endeavour to answer:

**How much money should we expect a data professional will make?**

This is a vague question that we will want to focus in on later, but at least it gives us a start.

## Data Processing

There are three steps to data processing:  data gathering, data cleansing, and data analysis.

### Data Gathering

In this example, we will stick to just the data professional survey. But if you want to take this further, a few additional data sources could be:

* PPP GDP per capita to normalize salaries across countries.
* A geocoding data set to visualize results on a map.
* Cost of living by ZIP code to normalize salaries within the US.
* Census information to build out data by ZIP code.
* Data from other surveys to add more to the sample.

If you are interested in extending this to include PPP GDP per capita, check out [a blog post I did based on the 2017 survey results](https://36chambers.wordpress.com/2017/01/18/analyzing-dba-salaries/).

### Data Cleansing

Here is where we begin the real work.  In the next snippet, I am going to load a series of libraries in R.  We will use each of them over the course of this notebook.  The `tidyverse` package is a series of incredibly useful libraries in R, and I can't think of doing a data science project in R without it.  The `XLConnect` package lets me read an Excel workbook easily and grab the salary data without much hassle.  The `caret` library provides some helpful tooling for working with data, including splitting out test versus training data, like we'll do below.  The `recipes` package will be useful for normalizing data later, and we will use `data.table` to get a glimpse at some of our uneven data.  We need the `devtools` package to install `keras` from GitHub.  Keras is a deep learning library which implements several neural network libraries, including TensorFlow, which we are using today.  We need to install TensorFlow on our machine.  Because this is a small data set, and because I want this to run on machines without powerful GPUs, I am using the CPU-based version of TensorFlow.  Performance should still be adequate for our purposes.

In [ ]:
if(!require(tidyverse)) {
  install.packages("tidyverse", repos = "http://cran.us.r-project.org")
  library(tidyverse)
}

if(!require(XLConnect)) {
  install.packages("XLConnect", repos = "http://cran.us.r-project.org")
  library(XLConnect)
}

if(!require(caret)) {
  install.packages("caret", repos = "http://cran.us.r-project.org")
  library(caret)
}

if(!require(recipes)) {
  install.packages("recipes", repos = "http://cran.us.r-project.org")
  library(caret)
}

if(!require(data.table)) {
  install.packages("data.table", repos = "http://cran.us.r-project.org")
  library(data.table)
}

if(!require(devtools)) {
  install.packages("devtools", repos = "http://cran.us.r-project.org")
  library(devtools)
}

if(!require(keras)) {
  devtools::install_github("rstudio/keras")
  library(keras)
  install_keras(method = "auto", conda = "auto", tensorflow = "default", extra_packages = NULL)
}

Once we have the required packages loaded, we will then load the Excel workbook.  I have verified the Excel worksheet and data region are correct, so we can grab the survey from the current directory and load it into `salary_data`.

In [ ]:
wb <- XLConnect::loadWorkbook("2018_Data_Professional_Salary_Survey_Responses.xlsx")
salary_data <- XLConnect::readWorksheet(wb, sheet = "Salary Survey", region = "A4:Z6015")

We can use the `glimpse` function inside the tidyverse to get a quick idea of what our `salary_data` dataframe looks like.  In total, we have 6011 observations of 26 variables, but this covers two survey years:  2017 and 2018.  Looking at the variable names, we can see that there are some which don't matter very much (like Timestamp, which is when the user filled out the form; and Counter, which is just a 1 for each record.

In [ ]:
glimpse(salary_data)

Our first data cleansing activity will be to filter our data to include just 2018 results, which gives us a sample size of 3,113 participants.  There are also results for 2017, but they asked a different set of questions and we don't want to complicate the analysis or strip out the new 2018 questions.

In [ ]:
survey_2018 <- filter(salary_data, Survey.Year == 2018)
nrow(survey_2018)

Looking at the survey, there are some interesting data points that we want:
* SalaryUSD (our label, that is, what we are going to try to predict)
* Country
* YearsWithThisDatabase
* EmploymentStatus
* JobTitle
* ManageStaff
* YearsWithThisTypeOfJob
* OtherPeopleOnYourTeam
* DatabaseServers
* Education
* EducationIsComputerRelated
* Certifications
* HoursWorkedPerWeek
* TelecommuteDaysPerWeek
* EmploymentSector
* LookingForAnotherJob
* CareerPlansThisYear
* Gender

For each of these variables, we want to see the range of options and perform any necessary cleanup.  The first thing I'd look at is the cardinality of each variable, followed by a detailed anlaysis of the smaller ones.

PrimaryDatabase is another variable which looks interesting, but it skews so heavily toward SQL Server that there's more noise than signal to it.  Because there are so many platforms with 10 or fewer entries and about 92% of entrants selected SQL Server, we'll throw it out.

In [ ]:
rapply(survey_2018, function(x) { length(unique(x)) })

In [ ]:
unique(survey_2018$Country)

In [ ]:
unique(survey_2018$EmploymentStatus)

We can use the `setDT` function on `data.table` to see just how many records we have for each level of a particular factor.  For example, we can see the different entries for PrimaryDatabase and EmploymentSector below.  Both of these are troublesome for our modeling because they both have a number of levels with 1-2 entries.  This makes it likely that we will fail to collect a relevant record in our training data set, and that will mess up our model later.  To rectify this, I am going to remove PrimaryDatabase as a feature and remove the two students from our sample.

In [ ]:
data.table::setDT(survey_2018)[, .N, keyby=PrimaryDatabase]

In [ ]:
data.table::setDT(survey_2018)[, .N, keyby=EmploymentSector]

Most of these columns came from dropdown lists, so they're already fairly clean.  But there are some exceptions to the rule.  They are:
* SalaryUSD
* YearsWithThisDatabase
* YearsWithThisTypeOfJob
* DatabaseServers
* HoursWorkedPerWeek
* Gender

All of these were text fields, and whenever a user gets to enter text, you can assume that something will go wrong.  For example:

In [ ]:
survey_2018 %>%
  distinct(YearsWithThisDatabase) %>%
  arrange(desc(YearsWithThisDatabase)) %>%
  top_n(10, YearsWithThisDatabase)

Someone with 53,716 years working with their primary database of choice?  That's committment!  You can also see a couple of people who clearly put in the year they started rather htan the number of years working with it, and someone who maybe meant 10 years?  But who knows, people type in weird stuff.

Anyhow, let's see how much that person with at least 10 thousand years of experience makes:

In [ ]:
survey_2018 %>%
  filter(YearsWithThisDatabase > 10000)

That's pretty sad, considering their millennia of work experience.  $95-98K isn't even that great a number.

Looking at years of experience with their current job roles, people tend to be more reasonable:

In [ ]:
survey_2018 %>%
  distinct(YearsWithThisTypeOfJob) %>%
  arrange(desc(YearsWithThisTypeOfJob)) %>%
  top_n(10, YearsWithThisTypeOfJob)

Next up, we want to look at the number of database servers owned.  500,000+ database servers is a bit excessive.  Frankly, I'm suspicious about any numbers greater than 5000, but because I can't prove it otherwise, I'll leave them be.

In [ ]:
survey_2018 %>%
  distinct(DatabaseServers) %>%
  arrange(desc(DatabaseServers)) %>%
  top_n(10, DatabaseServers)

survey_2018 %>%
  filter(DatabaseServers >= 5000) %>%
  arrange(desc(DatabaseServers))

The first entry looks like bogus data:  a $650K salary, a matching postal code, and 500K database servers, primarily in RDS?  Nope, I don't buy it.

The rest don't really look out of place, except that I think they put in the number of **databases** and not **servers**.  For these entrants, I'll change the number of servers to the median to avoid distorting things.

Now let's look at hours per week:

In [ ]:
survey_2018 %>%
  distinct(HoursWorkedPerWeek) %>%
  arrange(desc(HoursWorkedPerWeek)) %>%
  top_n(10, HoursWorkedPerWeek)

To the person who works 200 hours per week:  find a new job.  Your ability to pack more than 7\*24 hours of work into 7 days is too good to waste on a job making just $120K per year.

In [ ]:
survey_2018 %>%
  filter(HoursWorkedPerWeek >= 168) %>%
  arrange(desc(HoursWorkedPerWeek))

As far as Gender goes, there are only three with enough records to be significant:  Male, Female, and Prefer not to say.  We'll take Male and Female and bundle the rest under "Other" to get a small but not entirely insignificant set there.

In [ ]:
survey_2018 %>%
  group_by(Gender) %>%
  summarize(n = n())

In [ ]:
survey_2018 %>%
  group_by(Country) %>%
  summarize(n = n()) %>%
  filter(n >= 20)

There are only fifteen countries with at least 20 data points and just eight with at least 30.  This means that we won't get a great amount of information from cross-country comparisons outside of the sample.  Frankly, I might want to limit this to just the US, UK, Canada, and Australia, as the rest are marginal, but for this survey analysis, I'll keep the other eleven.

#### Building Our Cleaned-Up Data Set

Now that we've performed some basic analysis, we will clean up the data set.  I'm doing most of the cleanup in a single operation, but I do have some comment notes here, particularly around the oddities with SalaryUSD.  The SalaryUSD column has a few problems:
* Some people put in pennies, which aren't really that important at the level we're discussing.  I want to strip them out.
* Some people put in delimiters like commas or decimal points (which act as commas in countries like Germany).  I want to strip them out, particularly because the decimal point might interfere with my analysis, turning 100.000 to $100 instead of $100K.
* Some people included the dollar sign, so remove that, as well as any spaces.

It's not a perfect regex, but it did seem to fix the problems in this data set at least.

In [ ]:
valid_countries <- survey_2018 %>%
                    group_by(Country) %>%
                    summarize(n = n()) %>%
                    filter(n >= 20)

# Data cleanup
survey_2018 <- salary_data %>%
  filter(Survey.Year == 2018) %>%
  filter(HoursWorkedPerWeek < 200) %>%
  # There were only two students in the survey, so we will exclude them here.
  filter(EmploymentSector != "Student") %>%
  inner_join(valid_countries, by="Country") %>%
  mutate(
    SalaryUSD = stringr::str_replace_all(SalaryUSD, "\\$", "") %>%
      stringr::str_replace_all(., ",", "") %>%
      stringr::str_replace_all(., " ", "") %>%
      # Some people put in pennies.  Let's remove anything with a decimal point and then two numbers.
      stringr::str_replace_all(., stringr::regex("\\.[0-9]{2}$"), "") %>%
      # Now any decimal points remaining are formatting characters.
      stringr::str_replace_all(., "\\.", "") %>%
      as.numeric(.),
    # Some people have entered bad values here, so set them to the median.
    YearsWithThisDatabase = case_when(
      (YearsWithThisDatabase > 32) ~ median(YearsWithThisDatabase),
      TRUE ~ YearsWithThisDatabase
    ),
    # Some people apparently entered number of databases rather than number of servers.
    DatabaseServers = case_when(
      (DatabaseServers >= 5000) ~ median(DatabaseServers),
      TRUE ~ DatabaseServers
    ),
    EmploymentStatus = as.factor(EmploymentStatus),
    JobTitle = as.factor(JobTitle),
    ManageStaff = as.factor(ManageStaff),
    OtherPeopleOnYourTeam = as.factor(OtherPeopleOnYourTeam),
    Education = as.factor(Education),
    EducationIsComputerRelated = as.factor(EducationIsComputerRelated),
    Certifications = as.factor(Certifications),
    TelecommuteDaysPerWeek = as.factor(TelecommuteDaysPerWeek),
    EmploymentSector = as.factor(EmploymentSector),
    LookingForAnotherJob = as.factor(LookingForAnotherJob),
    CareerPlansThisYear = as.factor(CareerPlansThisYear),
    Gender = as.factor(case_when(
      (Gender == "Male") ~ "Male",
      (Gender == "Female") ~ "Female",
      TRUE ~ "Other"
    ))
  ) 

Now we can pare out variables we don't need.  Some of these, like postal code, are interesting but we just don't have enough data for it to make sense.  Others, like Kinds of Tasks Performed or Other Job Duties, have too many varieties for us to make much sense with a first pass.  They might be interesting in a subsequent analysis, though.

In [ ]:
survey_2018 <- survey_2018 %>%
  # One person had a salary of zero.  That's just not right.
  filter(SalaryUSD > 0) %>%
  select(-Counter, -KindsOfTasksPerformed, -OtherJobDuties, -OtherDatabases, -Timestamp, -Survey.Year, 
         -PostalCode, -n, -PrimaryDatabase)

Now that we have our salary data fixed, we can finally look at outliers.  I'd consider a salary of \$500K a year to be a bit weird for this field.  It's not impossible, but I am a little suspicious.  I am very suspicious of the part-timer making \$1.375 million, the federal employee making \$1 million, or the New Zealander making \$630K at a non-profit.

I'm kind of taking a risk by removing these, 

In [ ]:
survey_2018 %>%
  filter(SalaryUSD > 500000) %>%
  arrange(desc(SalaryUSD))

On the other side, there are 12 people who say they earned less than $5K a year.  Those also seem wrong.  Some of them look like dollars per  hour, and maybe some are monthly salary.  I'm going to strip those out.

In [ ]:
survey_2018 %>%
  filter(SalaryUSD < 5000) %>%
  arrange(desc(SalaryUSD))

In [ ]:
survey_2018 <- filter(survey_2018, SalaryUSD >= 5000 & SalaryUSD <= 500000)

### Data Analysis

We did some of the data analysis up above.  We can do additional visualization and correlation studies.  For example, let's look at a quick distribution of salaries after our cleanup work:

In [ ]:
summary(survey_2018$SalaryUSD)

We can also build a histogram pretty easily using the `ggplot2` library.  This shows the big clump of database professionals earning beween \$70K and \$115K per year.  This salary distribution does skew right a bit, as you can see.

In [ ]:
ggplot(data = survey_2018, mapping = aes(x = SalaryUSD)) +
  geom_histogram() +
  theme_minimal() +
  scale_x_log10(label = scales::dollar)

We can also break this down to look by primary job title, though I'll limit to a couple of summaries instead of showing a full picture.

In [ ]:
survey_2018 %>% filter(JobTitle == "Data Scientist") %>% select(SalaryUSD) %>% summary(.)

In [ ]:
survey_2018 %>% filter(JobTitle == "Developer: App code (C#, JS, etc)") %>% select(SalaryUSD) %>% summary(.)

In [ ]:
survey_2018 %>% filter(JobTitle == "Developer: T-SQL") %>% select(SalaryUSD) %>% summary(.)

This fit pretty well to my biases, although the max Data Scientist salary seems rather low.

## Modeling

Because our question is a "how much?" question, we want to use regression to solve the problem.  The most common form of regression that you'll see in demonstrations is linear regression, because it is easy to teach and easy to understand.  In today's demo, however, we're going to build a neural network with Keras.  Although our demo is in R, Keras actually uses Python on the back end to run TensorFlow.  There are other libraries out there which can run neural networks strictly within R (for example, Microsoft Machine Learning's R implemenation has the `RxNeuralNet()` function), but we will use Keras in this demo because it is a popular library.

Now that we have an algorithm and implementation in mind, let's split the data out into training and test subsets.  I want to use Country as the partition variable because I want to ensure that we retain some data from each country in the test set.  To make this split, I am using the `createDataPartition()` function in `caret`.  I'll then split out the data into training and test data.

In [ ]:
trainIndex <- caret::createDataPartition(survey_2018$Country, p = 0.7, list = FALSE, times = 1)
train_data <- survey_2018[trainIndex,]
test_data <- survey_2018[-trainIndex,]
nrow(train_data)
nrow(test_data)

Once I have this data split, I want to perform some operations on the training data.  Specifically, I want to do the following:
* One-Hot Encode the categorical data
* Mean-center the data, so that the mean of each numeric value is 0
* Scale the data, so that the standard deviation of each value is 1

The bottom two are called **normalizing** the data.  This is a valuable technique when dealing with many algorithms, including neural networks, as it helps with optimizing gradient descent problems.

In order to perform all of these operations, I will create a `recipe`, using the `recipes` package.

In [ ]:
rec_obj <- recipes::recipe(SalaryUSD ~ ., data = train_data) %>%       # Build out a set of rules we want to follow (a recipe)
  step_dummy(all_nominal(), -all_outcomes()) %>%              # One-hot encode categorical data
  step_center(all_predictors(), -all_outcomes()) %>%          # Mean-center the data
  step_scale(all_predictors(), -all_outcomes()) %>%           # Scale the data
  prep(data = train_data)

rec_obj

Now we can `bake` our data based on the recipe above.  Note that I performed all of these operations only on the **training** data.  If we normalize the training + test data, our optimization function can get a sneak peek at the distribution of the test data based on what is in the training set, and that will bias our result.

After building up the `x_` series of data sets, I'll build vectors which contain the salaries for the training and test data.  I need to make sure to remove the SalaryUSD variable; we don't want to make that available to the trainer as an independent variable!

In [ ]:
x_train_data <- recipes::bake(rec_obj, newdata = train_data)
x_test_data <- recipes::bake(rec_obj, newdata = test_data)
y_train_vec <- pull(x_train_data, SalaryUSD)
y_test_vec  <- pull(x_test_data, SalaryUSD)
# Remove the SalaryUSD variable.
x_train_data <- x_train_data[,-1]
x_test_data <- x_test_data[,-1]

At this point, I want to build the Keras model.  I'm creating a `build_model` function in case I want to run this over and over. In a real-life scenario, I would perform various optimizations, do cross-validation, etc.  In this scenario, however, I am just going to run one time against the full training data set, and then evaluate it against the test data set.

Inside the function, we start by declaring a Keras model.  Then, I add three layers to the model.  The first layer is a dense (fully-connected) layer which accepts the training data as inputs and uses the Rectified Linear Unit (ReLU) activation mechanism.  This is a decent first guess for activation mechanisms.  We then have a dropout layer, which reduces the risk of overfitting on the training data.  Finally, I have a dense layer for my output, which will give me the salary.

I compile the model using the `RMSProp` optimizer.  This is a good default optimizer for neural networks, although you might try `Adagrad`, `Adam`, or `AdaMax` as well.  Our loss function is Mean Squared Error, which is good for dealing with finding the error in a regression.  Finally, I'm interested in the Mean Absolute Error--that is, the dollar amount difference between our function's prediction and the actual salary.  The closer to \$0 this is, the better.

In [ ]:
build_model <- function() {
  model <- keras_model_sequential() %>%
    layer_dense(units = ncol(x_train_data), input_shape = c(ncol(x_train_data)), activation = "relu") %>%
    layer_dropout(rate = 0.05) %>%
    layer_dense(units = 1) # No activation --> linear layer

  # RMSProp is a nice default optimizer for a neural network.
  # Mean Squared Error is a classic loss function for dealing with regression-style problems, whether with a neural network or otherwise.
  # Mean Average Error gives us a metric which directly translates to the number of dollars we are off with our predictions.
  model %>% compile(
    optimizer = "rmsprop",
    loss = "mse",
    metrics = c("mae")
  )
}

Building out this model can take some time, so be patient.

In [ ]:
model <- build_model()
model %>% fit(as.matrix(x_train_data), y_train_vec, epochs = 30, batch_size = 16, verbose = 0)
result <- model %>% evaluate(as.matrix(x_test_data), y_test_vec)
result

What this tells us is that, after generating our model, we are an average of \$mean_absolute_error dollars off from reality.  In my case, that was over \$92K, so not very good at all.

With a real data science project, I would dig further, seeing if there were better algorithms available, cross-validating the training set, etc.  As-is, this result isn't good enough for a production scenario, but we can pretend that it is.

## Deployment

During the deployment phase, we will need to take our web service's inputs and transform them into the same shape as our training and test data.  We would then run the set and output our SalaryUSD prediction as the result.

I'm leaving the details of that as an exercise (because there are so many variables around deployment, including how you will fit it into your normal infrastructure), but a few thoughts:

1. As mentioned in the scenario, we're going to run a lot of one-off predictions rather than taking a batch of inputs, we want to optimize this for speed.  We could probably make this an asynchronous call so that when one of our users changes a detail like number of years working at the job, we can update our salary estimate.
2. This would probably be a good use case for SQL Server Machine Learning Services:  if we save all of the profile data in the database, we can pass those results straight to ML Services and perform the transformation and prediction in R.  If we go this route, we'd return the results as a SQL Server result set, which is easy for most developers to handle.
3. Another option would be to rewrite this in Python, as we're using Python under the covers anyhow.  Then we could use a web service framework like Flask to build a simple API.  If you go that route, you might want to stand up a production-grade service like Gunicorn and use Nginx as the front door to handle traffic and load balancing.

## Repeat The Process

From here, we would collect data on actual predictions.  We saw the test set's Mean Absolute Error, but an even more valuable measure is the Mean Absolute Error of actual data.  We'd need to ask people what their actual salaries are to know for sure, but we could feed these back into the training data and keep training our model.  One of the nice things about using a neural network like this is that we can continuously train the model based on our newly-gathered data.

Next, we would want to follow up with our stakeholders.  If they're happy with this result, we could probably look at other uses for the data source, such as cleaning up postal code and integrating with a service to get more granular information on salary in certain parts of larger countries like the US and Canada.